## Third Movement: 

In [24]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [25]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [26]:
%run attack_graph_MIR100.ipynb

Graph verification:
Number of nodes: 13
Number of edges: 13


### Use ??? Distribution

In [27]:
# The key change - enhanced random_steps function
attack_rate_list = [0]   # Not used but kept for API consistency
defense_rate_list = [0]  # Not used but kept for API consistency

def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    # THIS PART IS MISSING

In [28]:
%run ctr-core_simple.ipynb
#%run ctr-core_1.ipynb
main()

TypeError: cannot unpack non-iterable NoneType object

TypeError: cannot unpack non-iterable NoneType object

In [23]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Thu Dec 19 16:10:53 2024"

++++++++++++++++++++++++++++++++
attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 2.647059e-01
4 2.647059e-01
5 0.000000e+00
6 0.000000e+00
7 4.705883e-01

worst case attack strategies per goal:
          1
1 0.3676471
2 0.3676471
3 0.2647059
[1] 0.184

Defender can keep attacker success below: 0.184
Attacker can guarantee success probability of: 0.184

